**чатбот на рекуррентных нейросетях (Keras+TensorFlow)**

Евгений Борисов <borisov.e@solarl.ru>


----

## Библиотеки

In [5]:
import numpy as np
import re
import gzip
import pandas as pd
pd.options.display.max_colwidth = 200  


In [6]:
def pp(d): return "{:,.0f}".format(d).replace(",", " ")
def ppr(d): print('записей:', pp(len(d)) )  

## Учебные данные

In [7]:
with gzip.open('../data/text/rus-eng/rus.txt.gz','rt',encoding='utf8') as f: 
    pair = pd.DataFrame([ p.split('\t') for p in f.read().split('\n') if p.strip() ])

In [8]:
# with open('../data/text/pairs.txt','rt',encoding='utf8') as f: 
#     pair = pd.DataFrame([ p.split('%%') for p in f.read().split('\n') if p.strip() ])

In [9]:
pair.columns=['Q','A']
pair['Q'] = pair['Q'].str.strip()
pair['A'] = pair['A'].str.strip()

In [10]:
ppr(pair)
pair.sample(9)

записей: 336 666


,Q,A
301106,Can you believe that summer is almost over?,"Вы можете поверить, что лето уже почти прошло?"
1979,I screamed.,Я закричал.
301086,"At first, the local cowboys laughed at him.",Сначала местные ковбои над ним смеялись.
279086,"Tom tried to help us, but he couldn't.","Том пытался нам помочь, но не смог."
201835,Tom's situation was different.,У Тома была другая ситуация.
102234,Ten minus two is eight.,Десять минус два равно восемь.
122095,Can I cancel this ticket?,Могу ли я сдать обратно этот билет?
89594,Tell Tom I'm innocent.,"Скажите Тому, что я невиновен."
293418,I think you're putting in too much sugar.,"По-моему, вы кладёте слишком много сахара."


In [11]:
pair = pair.iloc[100000:110000]

ppr(pair)
# pair = pair.sample(1000)
# pair = pair.sample(283800)
# pair = pair.sample(600)

записей: 10 000


## Чистим тексты

In [12]:
pair['Q_clean'] = pair['Q'].str.lower()
pair['Q_clean'] = pair['Q_clean'].str.replace(r'([,.?!])', r' \1 ')
pair['A_clean'] = pair['A'].str.lower()
pair['A_clean'] = pair['A_clean'].str.replace(r'([,.?!])', r' \1 ')

# pair['A_clean'] = pair['A_clean'].apply(lambda s: re.sub( r'(\W)', ' \1 ', s))
# pair['A_clean'] = pair['A_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
# pair['A_clean'] = pair['A_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' digit ', s)) 

In [13]:
# # добавляем "служебные" слова - начало и конец последовательности
# pair['Q_clean'] = pair['Q_clean'].str.split() + ['<START>']
# pair['A_clean'] = ['<GO>'] + pair['A_clean'].str.split() + ['<EOS>']

In [14]:
pair['Q_clean'] = pair['Q_clean'].str.split()
pair['A_clean'] = pair['A_clean'].str.split()

In [15]:
pair[['Q_clean','A_clean']].sample(9)

,Q_clean,A_clean
104825,"[tom, put, on, a, black, wig, .]","[том, надел, чёрный, парик, .]"
100309,"[i'm, not, as, rich, as, tom, .]","[я, не, такой, богатый, ,, как, том, .]"
101889,"[she, pressed, the, switch, .]","[она, нажала, на, выключатель, .]"
104431,"[tom, is, wearing, sandals, .]","[том, в, сандалиях, .]"
102245,"[thank, you, for, the, help, .]","[спасибо, тебе, за, помощь, .]"
100328,"[i'm, not, going, to, do, it, .]","[я, не, собираюсь, этого, делать, .]"
103606,"[tom, did, twenty, pushups, .]","[том, сделал, 20, отжиманий, .]"
105103,"[tom, sprained, his, ankle, .]","[том, потянул, лодыжку, .]"
101231,"[let's, play, that, by, ear, .]","[давай, сыграем, это, на, слух, .]"


---

In [16]:
# считаем количество слов
pair['lenQ'] = pair['Q_clean'].str.len()
pair['lenA'] = pair['A_clean'].str.len()
pair.describe()

,lenQ,lenA
count,10000.000000,10000.000000
mean,5.722400,5.270400
std,0.737965,1.175084
min,3.000000,2.000000
25%,5.000000,4.000000
50%,6.000000,5.000000
75%,6.000000,6.000000
max,8.000000,11.000000


In [17]:
# определяем максимальную длинну последовательности
pair['lenQ'].quantile(0.95),  pair['lenA'].quantile(0.95)

(7.0, 7.0)

In [18]:
pair['Q_clean'] = pair['Q_clean'].apply( lambda t: list(reversed(t)) )

In [19]:
sent_len_a_max = pair['lenA'].max()
sent_len_q_max = pair['lenQ'].max()

## Кодируем тексты

In [20]:
%%time

from gensim.models.word2vec import Word2Vec

w2v_size = 128

CPU times: user 256 ms, sys: 3.05 ms, total: 259 ms
Wall time: 263 ms


In [21]:
w2v_q = Word2Vec( pair['Q_clean'].values.tolist(), min_count=1, size=w2v_size, window=4, workers=4)
w2v_q_vocab = sorted([w for w in w2v_q.wv.vocab])
ppr(w2v_q_vocab)

записей: 3 156


In [22]:
ii = np.random.permutation(len(w2v_q_vocab))[:10]
for i in ii:
    w = w2v_q_vocab[i]
    ww = [ v[0] for v in w2v_q.wv.most_similar(w, topn=5) ]
    print( w,':',ww )

lady : ['middle', 'dogs', 'first', 'ready', 'yesterday']
yours : ['!', "there's", 'than', 'likes', 'an']
diagnosis : ['station', 'puzzled', 'office', 'doubts', 'coca-cola']
funny : ['she', 'likes', 'his', 'no', 'down']
forgave : ['teasing', 'orders', 'armpits', 'abroad', 'china']
suffered : ['cup', 'idea', 'bed', 'doing', 'anymore']
aunt : ['seem', 'sure', 'soon', 'open', 'book']
fact : ['times', 'surprise', 'friend', 'contact', 'quite']
classmates : ['hanging', 'fingers', 'breakfast', 'boat', 'respect']
steps : ['kiss', 'am', 'pretty', 'message', 'choice']


/usr/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [23]:
w2v_a = Word2Vec( pair['A_clean'].values.tolist(), min_count=1, size=w2v_size, window=4, workers=4)
w2v_a_vocab = sorted([w for w in w2v_a.wv.vocab])
ppr(w2v_a_vocab)

записей: 6 545


In [24]:
ii = np.random.permutation(len(w2v_a_vocab))[:10]
for i in ii:
    w = w2v_a_vocab[i]
    ww = [ v[0] for v in w2v_a.wv.most_similar(w, topn=5) ]
    print( w,':',ww )

приходил : ['стол', 'приведи', 'заслуживает', 'ненадолго', 'грызть']
слушали : ['открывать', 'наконец', 'имеет', 'лекция', 'слышать']
обеспечен : ['встать', 'научите', 'морковь', 'книг', 'улицы']
даст : ['пострадать', 'кончились', 'начинаются', 'останусь', 'разобрал']
улыбнулась : ['твои', 'будем', 'зовут', 'помощь', 'где']
лестнице : ['сына', 'поесть', 'нужна', 'будем', 'убить']
умнее : ['очередь', 'мысли', 'любовь', 'сесть', 'одного']
ездит : ['слегка', 'номер', 'компьютер', 'что-нибудь', 'будь']
вине : ['спасла', 'вычистил', 'сохнет', 'выключен', 'поприветствовал']
вернёмся : ['включить', 'домашней', 'море', 'укусила', 'продажи']


---

In [25]:
pair['Q_code'] = pair['Q_clean'].apply(lambda t: [ w2v_q.wv.get_vector(w) for w in t ] )
pair['A_code'] = pair['A_clean'].apply(lambda t: [ w2v_a.wv.get_vector(w) for w in t ] )

In [26]:
# pair[['Q_code','A_code']].sample(3)


-----

In [27]:
# w2v_size = 128

PAD = np.zeros(w2v_size)

In [28]:
pair['Q_code'] = pair['Q_code'].apply( lambda t: [PAD]*(sent_len_q_max-len(t)) + t )

encoder_input_data = np.stack( pair['Q_code'].values ).astype(np.float32)

In [29]:
# encoder_input_data[1,-9:,:3]

In [30]:
GO = np.ones(w2v_size)*3
EOS = np.ones(w2v_size)*-3

sent_len_a_max+=2
pair['A_code'] = pair['A_code'].apply( lambda t: [GO] + t + [EOS] + [PAD]*(sent_len_a_max-len(t)-2)  )

In [31]:
decoder_input_data = np.stack( pair['A_code'].values )[:,:-1,:].astype(np.float32)
decoder_target_data = np.stack( pair['A_code'].values )[:,1:,:].astype(np.float32)

encoder_input_data.shape, decoder_input_data.shape, decoder_target_data.shape

((10000, 8, 128), (10000, 12, 128), (10000, 12, 128))

In [32]:
# decoder_input_data[2,:11,:3]

## Строим нейросеть

In [33]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [34]:
latent_dim = 256  # размер сети

In [35]:
encoder_inputs = Input(shape=(None, w2v_size))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [36]:
from keras import backend as K
def custom_activation(x):  return (K.tanh(x) * 4)

# model.add(Dense(32 , activation=custom_activation))

Using TensorFlow backend.


In [37]:
decoder_inputs = Input(shape=(None, w2v_size))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)

# decoder_dense = Dense(w2v_size)
# decoder_dense = Dense(w2v_size, activation='softmax')
# decoder_dense = Dense(w2v_size, activation='tanh')
# decoder_dense = Dense(w2v_size, activation='sigmoid')
decoder_dense = Dense(w2v_size, activation=custom_activation)

decoder_outputs = decoder_dense(decoder_outputs)

In [38]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [39]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 128)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 128)    0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 394240      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  394240      input_2[0][0]                    
                                                                 lstm[0][1]                       
          

In [40]:
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.compile(loss='mse', optimizer='rmsprop')
# model.compile(loss='mse', optimizer='adam')
# model.compile(loss='mse', optimizer='sgd')


Instructions for updating:
Use tf.cast instead.


In [41]:
%%time 

history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=100,
          epochs=50,
          validation_split=0.1
        )

Train on 9000 samples, validate on 1000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
9000/9000 [==============================] - 11s 1ms/sample - loss: 0.3096 - val_loss: 0.1659
Epoch 2/50
9000/9000 [==============================] - 10s 1ms/sample - loss: 0.1204 - val_loss: 0.1410
Epoch 3/50
9000/9000 [==============================] - 10s 1ms/sample - loss: 0.0912 - val_loss: 0.1305
Epoch 4/50
9000/9000 [==============================] - 10s 1ms/sample - loss: 0.0798 - val_loss: 0.1122
Epoch 5/50
9000/9000 [==============================] - 10s 1ms/sample - loss: 0.0710 - val_loss: 0.0946
Epoch 6/50
9000/9000 [==============================] - 10s 1ms/sample - loss: 0.0639 - val_loss: 0.0919
Epoch 7/50
9000/9000 [==============================] - 10s 1ms/sample - loss: 0.0573 - val_loss: 0.0787
Epoch 8/50
9000/9000 [==============================] - 10s 1ms/sample - loss: 0.0526 - val_loss: 0.0827
Epoch 9/50
9000/9000 [==============================] - 10s 1ms/

## Проверяем результат

In [42]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model( [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [45]:
def decode_sequence(input_seq):
    # генерируем состояние энкодера
    states_value = encoder_model.predict(input_seq)

    # вход декодера - последовательность из одного слова GO
    # output_w2v = w2v_a.wv['<GO>'].reshape([1,1,w2v_size])
    output_w2v = GO.reshape([1,1,w2v_size])

    # выходная последовательность
    decoded_sentence = []
    
    for i in range(sent_len_a_max): 
        output_w2v, h, c = decoder_model.predict([output_w2v] + states_value)
        
        # декодируем cлово
        cc = output_w2v.reshape(w2v_size)

        tt=cc-EOS
        ett = tt.dot(tt.T)/w2v_size
        
        # print(ett)
        # если очередное код слова это EOS
        if(ett<2.1): break # то завершаем работу
        
        w = w2v_a.wv.similar_by_vector(cc)[0][0] 
                
        decoded_sentence.append(w)

        # обновляем состояние сети
        states_value = [h, c]

    return ' '.join(decoded_sentence)

In [46]:
ii = np.random.permutation(len(encoder_input_data))[:10]
for seq_index in ii:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print( pair.iloc[seq_index]['Q'],' -> ', decoded_sentence )

You know what they say.  ->  получил рыбалку господина рыбалку плавать приняли пешком приняли скучал сильнее вслепую пойдёшь борту
I'm not sure I'm ready.  ->  слегка шанс рыбалку правильно рыбалку продолжать сильнее вслепую пойдёшь борту пробыли останется усну
Tom is driving me nuts.  ->  слегка следующего передал мою отдай дворе пойдёшь говорила пойдёшь входили останется водить начинали
Are you taller than Tom?  ->  вашего добьётся замечание дневник пловцов продолжать
Don't believe the media.  ->  получил приняли фотогеничны мою
Tell me you'll do that.  ->  скучает рыбалку рыбалку рыбалку рыбалку приняли видел рыбалку скучал пойдёшь разумно пробыли несовершеннолетний
Tom is in great danger.  ->  слегка кошка пойдёшь боитесь пора
I've got to keep going.  ->  получил рыбалку придёт французский туда рыбалку расскажи слегка столько приняли продолжать приняли вслепую
He loves no one but her.  ->  слегка кошка просите правильно рыбалку скучал приняли продолжать пойдёшь говорила
You knew I 

----

In [ ]:
# import matplotlib.pyplot as plt

In [ ]:
# history_dict = history.history
# history_dict.keys()

In [ ]:
# # acc = history.history['acc']
# #val_acc = history.history['val_acc']
# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(1, len(loss) + 1)
# plt.plot(epochs, loss, 'b', label='Training loss')
# plt.plot(epochs, val_loss, 'r', label='Validation loss')
# plt.title('Training and validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()

# plt.show()

In [ ]:
# plt.clf()   # clear figure
# acc_values = history_dict['acc']
# val_acc_values = history_dict['val_acc']

# plt.plot(epochs, acc, 'b', label='Training acc')
# plt.plot(epochs, val_acc, 'r', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()

# plt.show()

---

In [ ]:
# q_min = encoder_input_data.min()
# q_max = encoder_input_data.max()
# encoder_input_data = (encoder_input_data-q_min)/(q_max-q_min)

# q_fact = np.max( [np.abs(encoder_input_data.max()), np.abs(encoder_input_data.min())] )
# encoder_input_data = encoder_input_data/q_fact

In [ ]:
# encoder_input_data.min(),encoder_input_data.max()

In [ ]:
# a_min = decoder_input_data.min()
# a_max = decoder_input_data.max()
# decoder_input_data  = (decoder_input_data-a_min)/(a_max-a_min)
# decoder_target_data = (decoder_target_data-a_min)/(a_max-a_min)

# a_fact = np.max( [np.abs(decoder_input_data.max()), np.abs(decoder_input_data.min())] )
# decoder_input_data  = decoder_input_data/a_fact
# decoder_target_data = decoder_target_data/a_fact

In [ ]:
# decoder_target_data.min(),decoder_target_data.max()

In [ ]:
# PAD 

# x = np.array(range(1,7)).reshape([2,3])
# np.pad(x,[[0,3],[0,0]],'constant',constant_values=0)

# a = [[1, 2], [3, 4]]
# >>> np.pad(a, ((3, 2), (2, 3)), 'minimum')
# array([[1, 1, 1, 2, 1, 1, 1],
#        [1, 1, 1, 2, 1, 1, 1],
#        [1, 1, 1, 2, 1, 1, 1],
#        [1, 1, 1, 2, 1, 1, 1],
#        [3, 3, 3, 4, 3, 3, 3],
#        [1, 1, 1, 2, 1, 1, 1],
#        [1, 1, 1, 2, 1, 1, 1]])
# A = np.array([1,2,3,4,5])
# np.pad(A, (0, 3), 'constant')

In [ ]:
# # выбираем последовательности средней длинны
# # sent_len_min, sent_len_max = 7,10
# sent_len_min, sent_len_max = 5,12

# ppr(pair)
# pair = pair[
#     pair['lenQ'].between(sent_len_min,sent_len_max) 
#     & pair['lenA'].between(sent_len_min,sent_len_max) 
#   ]
# ppr(pair)

In [ ]:
# # выстраиваем входные последовательности в обратном порядке
# # и выравниваем длинну последовательностей,
# # дополняем короткие словом "служебным" словом,
# pad = ['<PAD>']*sent_len_q_max
# # pair['Q_clean'] = pair['Q_clean'].apply( lambda t: pad[len(t):] + list(reversed(t)) )
# pair['Q_clean'] = pair['Q_clean'].apply( lambda t: t + pad[len(t):] )

# pad = ['<PAD>']*sent_len_a_max
# pair['A_clean'] = pair['A_clean'].apply( lambda t: t + pad[len(t):] )

In [ ]:
# pair[['Q_clean','A_clean']].sample(9)

In [ ]:
# pair['A_code'] = pair['A_code'].apply( 
#     lambda t: np.pad( np.array([GO]+t+[EOS]),
#                       [[0,sent_len_a_max-len(t)+2,],[0,0]],
#                       mode='constant',
#                       constant_values=0) )

In [ ]:
# pair['Q_clean'] = pair['Q_clean'].apply( lambda t: t + pad[len(t):] )

# pair['Q_code'] = pair['Q_code'].apply( 
#     lambda t: np.pad( np.array(t),
#                       [[sent_len_q_max-len(t),0],[0,0]],
#                       mode='constant',
#                       constant_values=0) )

# encoder_input_data = np.stack( pair['Q_code'].values ).astype(np.float32)